In [2]:
!pip install python-dotenv
!pip install snowflake-connector-python

In [3]:
from dotenv import load_dotenv
import os
import pyspark
from pyspark.sql import SparkSession
import requests
import json
from pyspark.sql import functions as F
from pyspark.sql.types import StringType
import time

In [4]:
# Cargo mis variables de entorno
load_dotenv()

# Genero el dict de datos para conectarme con Snowflake a schema Raw
credencialesSnowflakeAnalytics = {
    "sfURL" : os.getenv("SNOWFLAKE_URL"),
    "sfUser" :  os.getenv("SNOWFLAKE_USER"),
    "sfPassword" : os.getenv("SNOWFLAKE_PASSWORD"),
    "sfDatabase" : os.getenv("SNOWFLAKE_DATABASE"),
    "sfSchema" : os.getenv("SNOWFLAKE_SCHEMA_ANALYTICS"),
    "sfWarehouse" : os.getenv("SNOWFLAKE_WAREHOUSE"),
    "sfRole" : os.getenv("SNOWFLAKE_ROLE"),
}

print(f"Estas son mis credenciales para Snowflake con schema Analytics: {credencialesSnowflakeAnalytics}")

Estas son mis credenciales para Snowflake con schema Analytics: {'sfURL': 'LSNDJXB-RHC82043.snowflakecomputing.com', 'sfUser': 'usuario_spark', 'sfPassword': 'EstudianteEstudiante64', 'sfDatabase': 'NY_TAXI', 'sfSchema': 'ANALYTICS', 'sfWarehouse': 'WAREHOUSE_TAXIS', 'sfRole': 'rol_pocos_privilegios'}


In [5]:
# Creo SparkSession para conexión con Snowflake
spark = (SparkSession.builder.appName("IngestaNewYorkTaxis").config("spark.jars.packages", "net.snowflake:snowflake-jdbc:3.13.30,net.snowflake:spark-snowflake_2.12:2.9.0-spark_3.1").getOrCreate())

print(spark)
print("Spark Version : " + spark.version)

# Ejecuto una query de prueba para validar comunicacion con Snowflake
query = "SELECT current_version()"

df = spark.read.format("snowflake").options(**credencialesSnowflakeAnalytics).option("query", query).load()

df.show()

Spark Version : 3.5.0
+-------------------+
|"CURRENT_VERSION()"|
+-------------------+
|             9.32.1|
+-------------------+



In [6]:
def generar_tabla_con_validaciones():

    print("Iniciando proceso de carga de datos de la OBT desde Schema Analytics para su validacion")

    try:
        df_obt = spark.read.format("snowflake") \
                .options(**credencialesSnowflakeAnalytics) \
                .option("dbtable", "NY_TAXI_OBT") \
                .load()

        df_obt_sin_nulos = df_obt.filter(F.col("DO_LOCATION_ID").isNotNull() & F.col("PASSENGER_COUNT").isNotNull() & F.col("PAYMENT_TYPE").isNotNull() & F.col("PU_LOCATION_ID").isNotNull() & F.col("RATE_CODE_ID").isNotNull() & F.col("DROPOFF_DATETIME").isNotNull() & F.col("PICKUP_DATETIME").isNotNull() & F.col("TRIP_DISTANCE").isNotNull() & F.col("VENDOR_ID").isNotNull())

        df_con_datos_coherentes = df_obt_sin_nulos.filter((F.col("PASSENGER_COUNT")>0) & (F.col("PASSENGER_COUNT")<10) & (F.col("EXTRA")>=0) & (F.col("FARE_AMOUNT")>=0) & (F.col("METROPOLITAN_TAX")>=0) & (F.col("TIP_AMOUNT")>=0) & (F.col("TOLLS_AMOUNT")>=0) & (F.col("TOTAL_AMOUNT")>=0) & (F.col("TRIP_DISTANCE")>0) & (F.col("TRIP_DURATION_MIN")>1) & (F.col("TRIP_DURATION_MIN")<180) & (F.col("AVG_SPEED_MPH")>0) & (F.col("AVG_SPEED_MPH")<100) & (F.col("TIP_PCT")>=0) & (F.col("PU_LOCATION_ID").between(1, 265)) & (F.col("DO_LOCATION_ID").between(1, 265)))
        
        df_con_fechas_coherentes= df_con_datos_coherentes.filter((F.col("MONTH")>0) & (F.col("MONTH")<13) & (F.col("YEAR")>=2015) & (F.col("YEAR")<=2025))

        print("Tabla OBT con validaciones generada correctamente")
        
        return df_con_fechas_coherentes
        
    except Exception as e:
        print(f"No se pudo generar la tabla OBT de Taxis con validaciones: {e}")
        raise e

In [7]:
def guardar_con_reintentos(df, tabla, reintentos=3):
    
    for intento in range(reintentos):
        try:
            print(f"Intento {intento + 1} de {reintentos} para guardar en {tabla}")
            df.write.format("snowflake") \
                .options(**credencialesSnowflakeAnalytics) \
                .option("dbtable", tabla) \
                .option("parallelism", 16) \
                .mode("append") \
                .save()
            
            print(f"Datos agregados exitosamente a TABLA OBT con Validaciones")
            return True
            
        except Exception as e:
            print(f"Error en intento {intento + 1}: {str(e)[:200]}...")
            if intento < reintentos - 1:
                wait_time = 30 
                print(f"Esperando {wait_time} segundos antes del reintento")
                time.sleep(wait_time)
            else:
                print(f"Todos los intentos fallaron para {tabla}")
                raise e

def guardar_por_lotes(df, tabla, batch_size=500000):
    
    total_count = df.count()
    num_partitions = max(1, total_count // batch_size)
    
    print(f"Dividiendo {total_count} filas en {num_partitions} particiones")
    
    df_reparticionado = df.repartition(num_partitions)
    
    return guardar_con_reintentos(df_reparticionado, tabla)

In [8]:
#Defino funciones tipicas de checkpoint para en caso de fallo no ingestar datos desde cero

def save_checkpoint(year, month, CHECKPOINT_FILE_COMBINADO):
    with open(CHECKPOINT_FILE_COMBINADO, "w") as f:
        json.dump({"year": year, "month": month}, f)

def load_checkpoint(CHECKPOINT_FILE_COMBINADO):
    if os.path.exists(CHECKPOINT_FILE_COMBINADO):
        with open(CHECKPOINT_FILE_COMBINADO, "r") as f:
            return json.load(f)
    return {"year": 0, "month": 0}

In [9]:
def cargar_tabla_obt_filtrada_snowflake():

    print("Iniciando proceso de generación de tabla OBT")
    
    try:
        tabla_destino="NY_TAXI_OBT_ULTIMATE"
        CHECKPOINT_FILE_OBT="checkpointCargaOBTUltimate.json"

        df_obt = generar_tabla_con_validaciones()
        print(df_obt.count())

        lista_years = sorted([int(item) for item in (os.getenv("YEARS").split(','))])
        lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
            
        checkpoint=load_checkpoint(CHECKPOINT_FILE_OBT)
        print(f"checkpoint: {checkpoint}")

        if ( checkpoint != {"year": 0, "month": 0} and (int(checkpoint["month"]) in lista_months) and (int(checkpoint["year"]) in lista_years)):    
            if ( int(checkpoint["month"]) == lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ):
                lista_years= lista_years[lista_years.index(checkpoint["year"])+1:]
            elif ( int(checkpoint["month"]) != lista_months[-1] and int(checkpoint["year"]) != lista_years[-1] ): 
                lista_years= lista_years[lista_years.index(checkpoint["year"]):]
                lista_months= lista_months[lista_months.index(checkpoint["month"])+1:]

        for year in lista_years:
            for month in lista_months:
                df_lote = df_obt.filter((F.col("YEAR") == int(year)) & (F.col("MONTH") == int(month)))
                print(f"Tabla OBT validada generada exitosamente para {month}-{year}.Se procedera a cargarlos en la base de Snowflake")
                guardar_por_lotes(df_lote, tabla_destino)
                print(f"Guardados correctamente datos OBT validados de taxis de year {year} + month {month}")
                save_checkpoint(year, month, CHECKPOINT_FILE_OBT)
            lista_months = sorted([int(item) for item in (os.getenv("MONTHS").split(','))])
        return True 
                
    except Exception as e2:
        print(f"Error generando o subiendo tabla OBT a Snowflake: {e2}")
        return False

In [10]:
cargar_tabla_obt_filtrada_snowflake()

Iniciando proceso de generación de tabla OBT
Iniciando proceso de carga de datos de la OBT desde Schema Analytics para su validacion
Tabla OBT con validaciones generada correctamente
813032013
checkpoint: {'year': 0, 'month': 0}
Tabla OBT validada generada exitosamente para 1-2015.Se procedera a cargarlos en la base de Snowflake
Dividiendo 14074381 filas en 28 particiones
Intento 1 de 3 para guardar en NY_TAXI_OBT_ULTIMATE
Error en intento 1: An error occurred while calling o161.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 19 in stage 9.0 failed 1 times, most recent failure: Lost task 19.0 in stage 9.0 (T...
Esperando 30 segundos antes del reintento
Intento 2 de 3 para guardar en NY_TAXI_OBT_ULTIMATE
Datos agregados exitosamente a TABLA OBT con Validaciones
Guardados correctamente datos OBT validados de taxis de year 2015 + month 1
Tabla OBT validada generada exitosamente para 2-2015.Se procedera a cargarlos en la base de Snowflake
Dividiendo 13844963

True